In [1]:
#Crawl 
import requests
import re
from bs4 import BeautifulSoup
from newspaper import Article
import urllib.request

# process 
import re
import pyvi
from pyvi import ViTokenizer #For split vietnamese words

#vector
from sklearn.feature_extraction.text import CountVectorizer

#
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Logistic
from sklearn.linear_model import LogisticRegression

#SVM
from sklearn.svm import SVC

In [2]:
phoneLinks = set()
r1 = requests.get('https://www.thegioididong.com/dtdd' )
coverpage = r1.content
soup1 = BeautifulSoup(coverpage, 'html.parser')
for a in soup1.find_all('a', href=True):
    if re.search('/dtdd/.*', a['href']):
        if a.find_all('div', class_='ratingresult'):
            phoneLinks.add(a['href'])
print(len(phoneLinks))

28


In [3]:
reviewLink=set()
for link in phoneLinks:
    if link.count('https://www.thegioididong.com')==0:
        link = 'https://www.thegioididong.com'+link
    link = link +'/danh-gia?p='
    reviewLink.add(link)
    #print(link)
print(len(reviewLink))

28


In [4]:
reviews =[]
for link in reviewLink:
    i = 1
    while True:
        link = link+str(i)
        try:
            page = urllib.request.urlopen(link)
            soup = BeautifulSoup(page, 'html.parser')
            new_feeds = soup.find(
                'ul', class_='ratingLst').find_all('li')
            
            article = Article(link)
            article.download()
            article.parse()
            soup = BeautifulSoup(article.html, 'html.parser')
            html = soup.findAll("div", {"class": "rc"})
            reviews = reviews + html
        
        except:
            break;
        i = i+1
print(len(reviews))


1753


In [5]:
comments = []
stars = []
labels = []
for li in reviews:
        rev = str(li).split('\n')
        if len(rev)>9:
            comment = rev[9].replace('<i>','').replace('</i>','')
            if comments.count(comment)==0:
                comments.append(comment)
                star = str(li).count('txtstar')
                stars.append(star)
                if star>3:
                    labels.append(1)
                else: 
                    labels.append(0)
print(len(comments))

746


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    comments, labels, train_size = 0.5
)

C:\Users\Thuy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
# ================== process raw data ===========

REPLACE_NO_SPACE = re.compile("[.;:\',\"()\[\]]")#("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def preprocess_reviews(reviews):
    #reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    reviews = [line.lstrip() for line in reviews]
    #reviews = [ViTokenizer.tokenize(rev) for rev in reviews]
    return reviews

X_train = preprocess_reviews(X_train)
X_test = preprocess_reviews(X_test)

In [9]:
cv = CountVectorizer(binary=True)
cv.fit(comments)

X_train = cv.transform(X_train)
X_test = cv.transform(X_test)

In [10]:

def SVM():
    clf = SVC(kernel = 'linear', C = 1e5) # just a big number 
    clf.fit(X_train.toarray(), y_train) 

    w = clf.coef_
    b = clf.intercept_
    #print('w = ', w)
    print('b = ', b)
    
    print ("Final Accuracy: %s"% accuracy_score(y_test, clf.predict(X_test.toarray())))
SVM()

b =  [-0.99960038]
Final Accuracy: 0.7667560321715817


In [11]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred_train = clf.predict(X_train)

y_pred_test = clf.predict(X_test)

acc_test = accuracy_score(y_pred_test,y_test)

acc_train = accuracy_score(y_pred_train,y_train)

print('accuracy_train: ', acc_train*100)

print('accuracy_test: ', acc_test*100)

accuracy_train:  94.36997319034852
accuracy_test:  82.57372654155496


In [12]:
from sklearn.linear_model import Perceptron

def perceptron():
    clf = Perceptron(tol=1e-3, random_state=0)
    clf.fit(X_train, y_train)
    acc_train=clf.score(X_train, y_train)
    acc_test =clf.score(X_test, y_test)
    #print('accurracy_train of folder {}: ', acc_train)
    print('accurracy_test perceptron: ', acc_test)

In [13]:
X_trainlog, X_testlog, y_trainlog, y_testlog = train_test_split(
    X_train, y_train, train_size = 0.75
)
X_trainlog = preprocess_reviews(X_trainlog)
X_testlog = preprocess_reviews(X_testlog)

X_trainlog = cv.transform(X_trainlog)
X_testlog = cv.transform(X_testlog)


def logistic():
    for c in [0.01, 0.05, 0.25, 0.5, 1]:
        lr = LogisticRegression(C=c)
        lr.fit(X_trainlog, y_trainlog)
        
        #print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_testlog, lr.predict(X_testlog))*100))
        
    final_model = LogisticRegression(C=0.05)
    final_model.fit(X_train, y_train)
    print ("Final Accuracy logistic: %s"% accuracy_score(y_test, final_model.predict(test)))


C:\Users\Thuy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


TypeError: expected string or bytes-like object

In [14]:
with open('data.txt','w',encoding="utf8") as f:
    f.write('\n'.join(comments))
    f.close()

In [ ]:
# ========== SORT ====================
zipped_pairs = zip(labels,comments ) 
comments = [x for _, x in sorted(zipped_pairs)] 
labels = sorted(labels)

an_iterator = filter(lambda number: number ==0, labels)
numNeg = len(list(an_iterator))
print(numNeg)

In [ ]:
#=============== shuffle ================
import random
random.seed(2)
random.shuffle(labels)
random.seed(2)
random.shuffle(comments)